In [302]:
# Чтобы скачать и распаковать zip-архив в output папку в Питоне:
import pandas as pd
import numpy as np
from IPython.display import display
#!/usr/bin/env python
import io
import zipfile
import requests  # $ pip install requests

r = requests.get("https://lms.skillfactory.ru/assets/courseware/v1/8e23b4ad7fb8d49f0b9c67be84dc440c/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/ratings_movies.zip")
with r, zipfile.ZipFile(io.BytesIO(r.content)) as archive:
    archive.extractall('Data')

In [303]:
df = pd.read_csv('Data/ratings_movies.csv', sep=',')
df = df.drop('Unnamed: 0', axis=1)
display(df.head())
df['date'] = pd.to_datetime(df['date'])
# df['userId'] = df['userId'].astype('category')
# df['genres'] = df['genres'].astype('category')
df.info()


,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   userId   100836 non-null  int64         
 1   movieId  100836 non-null  int64         
 2   rating   100836 non-null  float64       
 3   date     100836 non-null  datetime64[ns]
 4   title    100836 non-null  object        
 5   genres   100836 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 4.6+ MB


In [304]:
# Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию get_year_release(arg).
# библиотека для регулярных выражений

import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return 0
 
# Задание 8.1
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска? 
    
df['year_release'] = df['title'].apply(get_year_release)
mask_0 = df['year_release'] == 0 
display(df[mask_0]['movieId'].count())
df.info()


18

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   userId        100836 non-null  int64         
 1   movieId       100836 non-null  int64         
 2   rating        100836 non-null  float64       
 3   date          100836 non-null  datetime64[ns]
 4   title         100836 non-null  object        
 5   genres        100836 non-null  object        
 6   year_release  100836 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 5.4+ MB


In [305]:
# Задание 8.2
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
# В качестве ответа запишите название этого фильма без указания года его выпуска.

mask_year_1999 = df['year_release'] == 1999
df_1999 =df[mask_year_1999]
type(df_1999.head())
df_grouped = df_1999.groupby(by=['movieId', 'title'], as_index=False)['rating'].mean().sort_values(by='rating').head(1)['title'].values
display(str(*df_grouped).replace(' (1999)', ''))

'Bloodsport: The Dark Kumite'

In [306]:
# Задание 8.3
# 0/1 point (graded)
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

mask_year_2010 = df['year_release'] == 2010
df_2010 =df[mask_year_2010]
type(df_2010.head())
df_grouped = df_2010.groupby(by=['genres'], as_index=False)['rating'].mean().sort_values(by='rating').head(1)['genres'].values
display(*df_grouped)

'Action|Sci-Fi'

In [307]:
# Задание 8.4
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) жанров (genres) фильмов? 
# В качестве ответа запишите идентификатор этого пользователя.

df.groupby('userId')['genres'].nunique().idxmax()
# display(df[df['userId'] == 599].info())

599

In [308]:
# Задание 8.5
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя.
# Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.

df.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by= ['mean', 'count'], ascending=[False, True]).idxmax()['mean']

53

In [309]:
# Задание 8.6
# Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), 
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

mask_year_2018 = df['year_release'] == 2018
# mask_count_rating = df.groupby(['genres'])['rating'].count() > 10
df_main = df[mask_year_2018].groupby('genres', as_index=True)['rating'].agg(['mean', 'count']).sort_values(by='count', ascending=False)
mask_count = df_main['count'] >10
df_main[mask_count].idxmax()['mean']

'Action|Adventure|Sci-Fi'

In [361]:
# Задание 8.7
# Добавьте в таблицу новый признак year_rating — год выставления оценки. 
df['year_rating'] = df['date'].dt.year.astype('category')
# df.info()
# Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра.
table = df.pivot_table(values='rating', index='genres', columns='year_rating', fill_value= 0, aggfunc='mean').round(2)

# Выберите верные варианты ответа, исходя из построенной таблицы:

# A За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure ни разу не получало среднюю оценку ниже 3.
# вывод части сводной таблицы
table.query('genres== ["Action|Adventure"]')
#False
# проверка минимального значения по жанру
df[df['genres'] == "Action|Adventure"].groupby(['genres', 'year_rating'], as_index=True)['rating'].mean().min()

# B Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX получил в 2010 году.(2018)
max(table.query('genres== ["Action|Adventure|Animation|Children|Comedy|IMAX"]')) == 2010
#False

# C Среди сочетаний жанров, получивших наивысшую среднюю оценку в 2018 году, есть сочетание Animation|Children|Mystery.

max_mean_2018 = df[df['year_rating'] == 2018].groupby('genres', as_index=True)['rating'].mean().max()
df[df[df['year_rating'] == 2018].groupby('genres', as_index=True)['rating'].mean() == 5].groupby('genres', as_index=True)['rating'].mean()
# D Для жанра Comedy прослеживается тенденция падения рейтинга с каждым годом (с 1996 по 2018).

C:\Users\Denis.kozarenko\AppData\Local\Temp\ipykernel_20820\3124531404.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df[df['year_rating'] == 2018].groupby('genres', as_index=True)['rating'].mean() == 5].groupby('genres', as_index=True)['rating'].mean()


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).